Main Code

Code for generating comparisons

In [15]:
import numpy as np
import matplotlib.pyplot as plt

# Set seed for reproducibility
np.random.seed(42)

def generate_comparisons(true_r, true_beta, M, N, K):
    """
    Generates synthetic pairwise comparisons with constraints:
    - true_r ∈ [0, 1]
    - true_beta ∈ [0, 1]
    """
    comparisons = []
    for _ in range(M):
        k = np.random.randint(K)  # Worker index
        a, b = np.random.choice(N, 2, replace=False)

        # Compute probability using constrained rewards and beta
        reward_diff = true_r[a] - true_r[b]
        prob = 1 / (1 + np.exp(-true_beta[k] * reward_diff))

        if np.random.rand() < prob:
            comparisons.append((a, b, k))
        else:
            comparisons.append((b, a, k))
    return comparisons

In [16]:
def metrics(true_r, true_beta, estimated_r, estimated_beta):
  rmse_r = np.sqrt(np.mean((true_r - estimated_r)**2))
  rmse_beta = np.sqrt(np.mean((true_beta - estimated_beta)**2))
  # print(f"RMSE for Item Rewards: {rmse_r}")
  return rmse_r, rmse_beta

EM Code

Experiments by varying:
1. Number of items.
2. Number of comparisons.
3. Number of workers.

Metrics:
1. Accuracy
2. Comparison with Majority vote
3. RMSE between competence and reward vector

In [3]:
def get_original_label(comparisons):
  original_labels = []
  for w_i, l_i, _ in comparisons:
    w_i = int(w_i)
    l_i = int(l_i)
    if true_r[w_i] > true_r[l_i]:
        original_labels.append(1)
    else:
        original_labels.append(0)
  return original_labels

In [4]:
def estimate_labels(estimated_r1):
  estimated_labels = []
  for w_i, l_i, _ in comparisons:
    w_i = int(w_i)
    l_i = int(l_i)
    if true_r[w_i] > true_r[l_i]:
      if estimated_r1[w_i] > estimated_r1[l_i]:
        estimated_labels.append(1)
      else:
        estimated_labels.append(0)
    else:
      if true_r[w_i] < true_r[l_i]:
        if estimated_r1[w_i] < estimated_r1[l_i]:
          estimated_labels.append(1)
        else:
          estimated_labels.append(0)
  return estimated_labels

In [14]:
import torch
from tqdm import tqdm

class PolyaGammaEMGPU:
    def __init__(self, num_items, num_workers, max_iter=500, epsilon=1e-4, device='cuda'):
        self.num_items = num_items
        self.num_workers = num_workers
        self.max_iter = max_iter
        self.epsilon = epsilon
        self.device = device

        # Initialize parameters with identifiability constraints
        self.r = torch.randn(num_items, device=device)
        self.r -= self.r.mean()
        self.beta = torch.empty(num_workers, device=device).uniform_(0.1, 0.9)
        
        # Precompute constants
        self.ones = torch.ones(1, device=device)
        self.half = torch.tensor(0.5, device=device)
        self.eps = torch.tensor(1e-10, device=device)

    def fit(self, comparisons):
        # Convert comparisons to tensor and move to device
        comp_tensor = torch.tensor(comparisons, dtype=torch.long, device=self.device)
        w = comp_tensor[:, 0]
        l = comp_tensor[:, 1]
        k = comp_tensor[:, 2]
        
        prev_r = self.r.clone()
        prev_beta = self.beta.clone()

        for _ in tqdm(range(self.max_iter)):
            # E-step: Vectorized PG expectations
            x = self.beta[k] * (self.r[w] - self.r[l])
            ew = torch.where(
                x.abs() < 1e-8,
                0.25 - (x**2)/48.0,
                torch.tanh(x/2) / (2*x)
            )

            # M-step: Vectorized updates
            self._update_rewards(w, l, k, ew)
            self._update_competencies(w, l, k, ew)

            # Enforce constraints
            self.r -= self.r.mean()
            self.beta = torch.clamp(self.beta, 1e-10, 1-1e-10)

            # Check convergence
            if self._check_convergence(prev_r, prev_beta):
                break
            prev_r.copy_(self.r)
            prev_beta.copy_(self.beta)

        return self.r.cpu().numpy(), self.beta.cpu().numpy()

    def _update_rewards(self, w, l, k, ew):
        beta_k = self.beta[k]
        beta_sq = beta_k ** 2
        
        # Vectorized numerator/denominator updates
        numerator = torch.zeros_like(self.r)
        denominator = torch.zeros_like(self.r)
        
        # Winning items
        numerator.scatter_add_(0, w, 0.5 * ew)
        denominator.scatter_add_(0, w, ew * beta_sq)
        
        # Losing items
        numerator.scatter_add_(0, l, -0.5 * ew)
        denominator.scatter_add_(0, l, ew * beta_sq)

        # Update rewards with stability
        self.r = numerator / torch.clamp(denominator, min=1e-10)

    def _update_competencies(self, w, l, k, ew):
        delta = self.r[w] - self.r[l]
        beta_k_old = torch.clamp(self.beta[k], min=1e-10)
        factor = 0.5 / beta_k_old
        
        # Vectorized updates
        numerator = torch.zeros_like(self.beta)
        denominator = torch.zeros_like(self.beta)
        
        numerator.scatter_add_(0, k, ew * factor * delta)
        denominator.scatter_add_(0, k, ew * delta ** 2)

        # Update only workers with comparisons
        mask = denominator > 1e-10
        self.beta[mask] = numerator[mask] / denominator[mask]

    def _check_convergence(self, prev_r, prev_beta):
        r_diff = torch.norm(self.r - prev_r)
        beta_diff = torch.norm(self.beta - prev_beta)
        return r_diff < self.epsilon and beta_diff < self.epsilon

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Set seed for reproducibility
np.random.seed(42)

N = 50                  # Number of items
K = 6                   # Number of worker
# model = GradientEM(num_items=N, num_workers=K, learning_rate=0.01, max_iter=500)

# True parameters (constrained to [0, 1])
true_r = np.random.rand(N)     # Item rewards (0 = worst, 1 = best)
true_r = true_r - true_r.mean()
true_beta = np.random.rand(K)          # Worker competencies (0 = random, 1 = perfect)
M_array = [100, 1000, 5000, 10000, 20000, 40000, 60000]
# M_array = [60000]
accuracy_scores = []
original_accuracy_scores = []
rmse_r_array = []
rmse_beta_array = []


for m in M_array:
    comparisons = generate_comparisons(true_r, true_beta, m, N, K)
    # estimated_r3, estimated_beta3, history3 = model.fit(comparisons)
    model = PolyaGammaEMGPU(N, K, device=device)
    r_est, b_est = model.fit(comparisons)

#     rmse_r3, rmse_beta3 = metrics(true_r, true_beta, r_est, b_est)
    #   print(rmse_r3)
    #   print(rmse_beta3)
    true_labels = [1]*len(comparisons)
    estimated_labels = estimate_labels(r_est)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, estimated_labels)
    accuracy_scores.append(accuracy)
    #   rmse_r_array.append(rmse_r3)
    #   rmse_beta_array.append(rmse_beta3)
    print(accuracy)


    original_labels = get_original_label(comparisons)
    original_accuracy = accuracy_score(true_labels, original_labels)
    print(original_accuracy)
    original_accuracy_scores.append(original_accuracy)

print(accuracy_scores)

NameError: name 'device' is not defined

In [7]:
!nvidia-smi

Fri May  2 21:54:56 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   71C    P0             237W / 300W |  44371MiB / 81920MiB |     70%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
def get_original_label(comparisons):
  original_labels = []
  for w_i, l_i, _ in comparisons:
    w_i = int(w_i)
    l_i = int(l_i)
    if true_r[w_i] > true_r[l_i]:
        original_labels.append(1)
    else:
        original_labels.append(0)
  return original_labels

In [11]:
def estimate_labels(estimated_r1):
  estimated_labels = []
  for w_i, l_i, _ in comparisons:
    w_i = int(w_i)
    l_i = int(l_i)
    if true_r[w_i] > true_r[l_i]:
      if estimated_r1[w_i] > estimated_r1[l_i]:
        estimated_labels.append(1)
      else:
        estimated_labels.append(0)
    else:
      if true_r[w_i] < true_r[l_i]:
        if estimated_r1[w_i] < estimated_r1[l_i]:
          estimated_labels.append(1)
        else:
          estimated_labels.append(0)
  return estimated_labels

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Set seed for reproducibility
np.random.seed(42)
N_array = [10, 50, 50, 100, 200, 100, 200, 200, 500]
K_array = [2, 2, 6, 8, 8, 10, 10, 50, 50]
# N_array = [50]
# K_array = [6]

scores_dict = {}

for N, K in zip(N_array, K_array):
    # True parameters (constrained to [0, 1])
    true_r = np.random.rand(N)     # Item rewards (0 = worst, 1 = best)
    true_r = true_r - true_r.mean()
    true_beta = np.random.rand(K)          # Worker competencies (0 = random, 1 = perfect)
    M_array = [100, 1000, 5000, 10000, 20000, 40000, 60000, 80000, 100000]
    # M_array = [60000]
    accuracy_scores = []
    original_accuracy_scores = []
    rmse_r_array = []
    rmse_beta_array = []


    for m in M_array:
        comparisons = generate_comparisons(true_r, true_beta, m, N, K)
        # estimated_r3, estimated_beta3, history3 = model.fit(comparisons)
        model = PolyaGammaEMGPU(N, K, device=device)
        r_est, b_est = model.fit(comparisons)
        
        r_est = 1 / (1 + np.exp(-r_est))
        r_est = r_est - r_est.mean()
        
        rmse_r3, rmse_beta3 = metrics(true_r, true_beta, r_est, b_est)
#           print(rmse_r3)
#           print(rmse_beta3)
        true_labels = [1]*len(comparisons)
        estimated_labels = estimate_labels(r_est)

        # Calculate accuracy
        accuracy = accuracy_score(true_labels, estimated_labels)
        accuracy_scores.append(accuracy)
        rmse_r_array.append(rmse_r3)
        rmse_beta_array.append(rmse_beta3)
#         print(accuracy)


        original_labels = get_original_label(comparisons)
        original_accuracy = accuracy_score(true_labels, original_labels)
#         print(original_accuracy)
        print(f"N={N}, K={K}, m={m}: EM={accuracy:.4f}, Original={original_accuracy:.4f}")
        original_accuracy_scores.append(original_accuracy)

    print(accuracy_scores)
    scores_dict[f'{N}_{K}'] = {
        "our": accuracy_scores,
        "original": original_accuracy_scores
    }

  2%|██▉                                                                                                                                              | 10/500 [00:00<00:00, 571.59it/s]


N=10, K=2, m=100: EM=0.5100, Original=0.5400


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1122.47it/s]


N=10, K=2, m=1000: EM=0.7140, Original=0.5510


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1155.30it/s]


...

N=500, K=50, m=60000: EM=0.7611, Original=0.5377


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1369.93it/s]


N=500, K=50, m=80000: EM=0.7870, Original=0.5385


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1326.45it/s]


N=500, K=50, m=100000: EM=0.8089, Original=0.5388
[0.39, 0.114, 0.61, 0.6188, 0.67505, 0.73215, 0.7611333333333333, 0.786975, 0.80891]


In [25]:
print(rmse_r_array)

[0.40357061062865135, 0.4638166410605034, 0.28076868788079207, 0.28155996546845796, 0.28119404362177763, 0.2803117800053404, 0.28067241988174596, 0.2800888948893563, 0.28013357316490994]


In [26]:
print(rmse_beta_array)

[0.4878252053129532, 0.5446387913793052, 0.4710255682501327, 0.49315283212723615, 0.4855037421730597, 0.46917884048399444, 0.48208020822112324, 0.4508856152784744, 0.4818467712243634]


In [13]:
scores_dict["50_6"]["our"]

KeyError: '50_6'

In [ ]:
import matplotlib.pyplot as plt

M_array = [100, 1000, 5000, 10000, 20000, 40000, 60000, 80000, 100000]
N_array = [10, 50, 50, 100, 200, 100, 200, 200, 500]
K_array = [2, 2, 6, 8, 8, 10, 10, 50, 50]

for N, K in zip(N_array, K_array):
    key = f"{N}_{K}"
    acc = scores_dict[key]["our"]
    original_acc = scores_dict[key]["original"]

    plt.figure(figsize=(8, 5))
    plt.plot(M_array, acc, marker='o', label='EM (ours)')
    plt.plot(M_array, original_acc, marker='x', linestyle='--', label='Original')
    
    plt.xlabel("Number of comparisons (m)")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy vs. m for N={N}, K={K}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

M_array = [100, 1000, 5000, 10000, 20000, 40000, 60000, 80000, 100000]
N_array = [10, 50, 50, 200, 200, 500]
K_array = [2, 2, 6, 10, 50, 50]

plt.figure(figsize=(12, 7))

i = 0
for N, K in zip(N_array, K_array):
    key = f"{N}_{K}"
    acc = scores_dict[key]["our"]
    original_acc = scores_dict[key]["original"]

    label_em = f"EM: N={N}, K={K}"
    label_orig = f"Original"
    
    plt.plot(M_array, acc, marker='o', label=label_em)
    if i == 0:
        plt.plot(M_array, original_acc, marker='x', linestyle='--', label=label_orig)
        i = i + 1
    else:
        plt.plot(M_array, original_acc, marker='x', linestyle='--')

plt.xlabel("Number of comparisons (m)")
plt.ylabel("Accuracy")
# plt.title("Performance of Extended BTL Model on various (N,K) settings")
plt.legend(loc='best', fontsize=8)
plt.grid(True)
plt.tight_layout()
plt.savefig("result_line_graph.svg", format="svg")